# 전국 시군구 전/출입 data
* 칼럼 생성
* 시도+시군구=> 행정구역 코드 생성
* 세종시 data 필터링 

In [7]:
import pandas as pd

In [8]:
# encoding='euc-kr'

path= '세대관련연간자료_2020_20210814_33953.csv'

# 세대관련연간자료의 용량문제로 다운시 목록명이 미포함 됨
# 파일 불러오면서 컬럼 생성해주기

df=pd.read_csv(path, encoding='cp949',names=['전입행정_시도','전입행정_시군구','전입행정_동읍면','전입년','전입원','전입일','전출행정_시도','전출행정_시군구','전출행정_동읍면','전입사유','세대관련','이동_총인구','인동_남인구','이동_여인구'], header=None)

In [9]:
df

,전입행정_시도,전입행정_시군구,전입행정_동읍면,전입년,전입원,전입일,전출행정_시도,전출행정_시군구,전출행정_동읍면,전입사유,세대관련,이동_총인구,인동_남인구,이동_여인구
0,11,110,51500,2020,1,2,11,110,51500,3,1,3,2,1
1,11,110,51500,2020,1,2,11,410,66000,2,1,4,2,2
2,11,110,51500,2020,1,2,11,710,72000,2,2,1,0,1
3,11,110,51500,2020,1,2,41,135,67000,2,2,1,0,1
4,11,110,51500,2020,1,3,11,110,51500,4,1,5,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6528499,50,130,62000,2020,12,28,41,463,51700,6,2,1,1,0
6528500,50,130,62000,2020,12,28,48,127,25000,1,2,1,1,0
6528501,50,130,62000,2020,12,28,50,110,54000,6,2,1,1,0
6528502,50,130,62000,2020,12,29,50,130,62000,9,2,1,1,0


* 여기서 합쳐진 시군구 코드는 문자열!!!!!!

In [18]:
# 각각의 전입/전출 시도/시군구 코드를 합쳐서 하나로 생성
df['in_cd']=df['전입행정_시도'].astype(str)+df["전입행정_시군구"].astype(str)
df['out_cd']=df['전출행정_시도'].astype(str)+df["전출행정_시군구"].astype(str)

df.tail(3)

,전입행정_시도,전입행정_시군구,전입행정_동읍면,전입년,전입원,전입일,전출행정_시도,전출행정_시군구,전출행정_동읍면,전입사유,세대관련,이동_총인구,인동_남인구,이동_여인구,in_cd,out_cd
6528501,50,130,62000,2020,12,28,50,110,54000,6,2,1,1,0,50130,50110
6528502,50,130,62000,2020,12,29,50,130,62000,9,2,1,1,0,50130,50130
6528503,50,130,62000,2020,12,30,50,130,62000,3,2,1,1,0,50130,50130


In [19]:
df_sum = df.groupby(['out_cd','in_cd'])['이동_총인구'].sum().reset_index()

In [20]:
df_sum

,out_cd,in_cd,이동_총인구
0,11110,11110,9631
1,11110,11140,563
2,11110,11170,648
3,11110,11200,448
4,11110,11215,274
...,...,...,...
58438,50130,48870,11
58439,50130,48880,13
58440,50130,48890,8
58441,50130,50110,5233


In [21]:
df_sum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58443 entries, 0 to 58442
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   out_cd  58443 non-null  object
 1   in_cd   58443 non-null  object
 2   이동_총인구  58443 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [22]:
# 전입지와 전출지가 같은 행은 삭제
# 시각화에서 곂쳐서 다른 요소 방해할까봐

df_sum_ind = df_sum[df_sum.out_cd==df_sum.in_cd].index

df_sum_result = df_sum.drop(index=df_sum_ind).reset_index(drop=True)
df_sum_result

,out_cd,in_cd,이동_총인구
0,11110,11140,563
1,11110,11170,648
2,11110,11200,448
3,11110,11215,274
4,11110,11230,735
...,...,...,...
58188,50130,48860,7
58189,50130,48870,11
58190,50130,48880,13
58191,50130,48890,8


In [23]:
# df_sum_result.to_csv('2020 시군구 이동 전출입 이동 통계.csv')

In [24]:
#전국에서 세종시 전/출입 데이터 필터링

df1=df_sum_result[(df_sum_result['out_cd'] == '36110') | (df_sum_result['in_cd'] == '36110')]

In [25]:
df1

,out_cd,in_cd,이동_총인구
73,11110,36110,140
320,11140,36110,83
562,11170,36110,190
806,11200,36110,181
1054,11215,36110,159
...,...,...,...
57108,48870,36110,11
57328,48880,36110,26
57549,48890,36110,14
57771,50110,36110,217


#### 파일 저장

In [26]:

#df1.to_csv('2020 세종시 전출입 이동 통계.csv',encoding='euc-kr')

In [27]:
a='https://flowmap.blue/1-ZTHWLp6NGiaLGf69rVh98488IoWWkZx2mZIb4bmQg8?v=36.472576,127.317271,7.46,0,0&a=0&as=1&b=1&bo=100&c=1&ca=1&cz=7&d=0&fe=1&lt=1&lfm=ALL&col=Viridis&f=0'
print('2020 세종시 전/출입 data 시각화', end=' ' )
print (a)

2020 세종시 전/출입 data 시각화 https://flowmap.blue/1-ZTHWLp6NGiaLGf69rVh98488IoWWkZx2mZIb4bmQg8?v=36.472576,127.317271,7.46,0,0&a=0&as=1&b=1&bo=100&c=1&ca=1&cz=7&d=0&fe=1&lt=1&lfm=ALL&col=Viridis&f=0


## 세종시 data + 행정구역 코드 합치기

In [30]:
code=pd.read_csv('전국 시군구 중심점 (1).csv',encoding='euc-kr')

In [31]:
code

,SIG_CD,SIG_KOR_NM,Y,X,SIG_ENG_NM
0,11110,종로구,37.594920,130.864168,Jongno-gu
1,11140,중구,37.560143,128.672075,Jung-gu
2,11170,용산구,37.531386,128.485244,Yongsan-gu
3,11200,성동구,37.551030,128.536437,Seongdong-gu
4,11215,광진구,37.546722,128.736281,Gwangjin-gu
...,...,...,...,...,...
236,48870,함양군,35.551727,NaN,Hamyang-gun
237,48880,거창군,35.732613,NaN,Geochang-gun
238,48890,합천군,35.576662,NaN,Hapcheon-gun
239,50110,제주시,33.442124,NaN,Jeju-si


### 
* 세종시 전출입 데이터 명= df1
* 전국 시군구 코드 데이터 명= code

In [32]:
name = code[['SIG_CD','SIG_KOR_NM']]
name.head(3)

,SIG_CD,SIG_KOR_NM
0,11110,종로구
1,11140,중구
2,11170,용산구


In [33]:
df1.head(2)

,out_cd,in_cd,이동_총인구
73,11110,36110,140
320,11140,36110,83


In [60]:
df2=df1.rename(columns={'out_cd':'전출도시','in_cd':'전입도시'})
df2

,전출도시,전입도시,이동_총인구
73,11110,36110,140
320,11140,36110,83
562,11170,36110,190
806,11200,36110,181
1054,11215,36110,159
...,...,...,...
57108,48870,36110,11
57328,48880,36110,26
57549,48890,36110,14
57771,50110,36110,217


In [63]:
# 세종에서 제일 많이 인구가 이동한 도시 추출 (세종에서 전출)

q=df2.loc[df2['전출도시']=='36110',]
q

,전출도시,전입도시,이동_총인구
17931,36110,11110,112
17932,36110,11140,99
17933,36110,11170,150
17934,36110,11200,163
17935,36110,11215,168
...,...,...,...
18175,36110,48870,12
18176,36110,48880,11
18177,36110,48890,3
18178,36110,50110,195


In [129]:
ins=q[q['전입도시'].str.startswith("50")]
ins['이동_총인구'].sum()

# 세종 -> 서울11 =  5107
# 세종 -> 부산26 =  600
# 세종 -> 대구 27=  511
# 세종 -> 인천 28=  901
# 세종 -> 대전 30= 8254
# 세종 -> 경기도41= 5728
# 세종 -> 청주4311= 4385
# 세종 -> 천안4413= 1393
# 세종 -> 공주 =  1412
# 세종 -> 논산 = 338

301

In [64]:
#세종으로 가장 많은 인구가 이동한 도시 추출 (세종으로의 전입)

qq=df2[df2['전입도시']=='36110']
qq

,전출도시,전입도시,이동_총인구
73,11110,36110,140
320,11140,36110,83
562,11170,36110,190
806,11200,36110,181
1054,11215,36110,159
...,...,...,...
57108,48870,36110,11
57328,48880,36110,26
57549,48890,36110,14
57771,50110,36110,217


In [128]:
outs=qq[qq['전출도시'].str.startswith("50")]
outs['이동_총인구'].sum()

# 서울에서 세종시로의 전출한 인구수 =5546

# 부산-세종 = 967
# 대구-세종 = 879
# 인천-세종 = 1117
# 대전-세종 = 15425
# 청주-세종 = 4190
# 천안-세종 = 1362
# 공주-세종 = 2029
# 논산-세종 = 509



308

###### q= 세종에서 가장 많이 인구가 전출간 도시 
* 세종에서의 전출

###### qq= 세종으로 가장 많이 인구가 전입온 도시
* 세종으로의 전입

In [97]:
q1=q.sort_values(by=['이동_총인구'],ascending=[False])
qq1=q1.head(13)
qq1['전입도시명']=['대전_유성구','청주_흥덕구','대전_서구','공주시','대전_대덕구','대전_중구','대전_동구','청주_서원구','천안_동남구','청주_상당구','천안_서북구','청주_청원구','평택시']
qq1

<ipython-input-97-e94b058f23bd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qq1['전입도시명']=['대전_유성구','청주_흥덕구','대전_서구','공주시','대전_대덕구','대전_중구','대전_동구','청주_서원구','천안_동남구','청주_상당구','천안_서북구','청주_청원구','평택시']


,전출도시,전입도시,이동_총인구,전입도시명
17998,36110,30200,3309,대전_유성구
18067,36110,43113,2431,청주_흥덕구
17997,36110,30170,2408,대전_서구
18081,36110,44150,1412,공주시
17999,36110,30230,895,대전_대덕구
17996,36110,30140,852,대전_중구
17995,36110,30110,790,대전_동구
18066,36110,43112,771,청주_서원구
18079,36110,44131,715,천안_동남구
18065,36110,43111,680,청주_상당구


In [99]:
q2=qq.sort_values(by=['이동_총인구'],ascending=[False])
qq2=q2.head(13)
qq2['전출도시명']=['대전_유성구','대전_서구','공주시','청주_흥덕구','대전_중구','대전_대덕구','대전_동구','청주_서원구','청주_청원구','천안_서북구','천안_동남구','청주_상당구','논산시']
qq2

<ipython-input-99-4fab804f6fdf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qq2['전출도시명']=['대전_유성구','대전_서구','공주시','청주_흥덕구','대전_중구','대전_대덕구','대전_동구','청주_서원구','청주_청원구','천안_서북구','천안_동남구','청주_상당구','논산시']


,전출도시,전입도시,이동_총인구,전출도시명
16282,30200,36110,6048,대전_유성구
16033,30170,36110,4940,대전_서구
36136,44150,36110,2029,공주시
32920,43113,36110,1826,청주_흥덕구
15786,30140,36110,1610,대전_중구
16530,30230,36110,1499,대전_대덕구
15540,30110,36110,1328,대전_동구
32678,43112,36110,999,청주_서원구
33165,43114,36110,822,청주_청원구
35890,44133,36110,727,천안_서북구


##### 전입지/전출지 코드_한글 ver.